In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import os
load_dotenv()
import yfinance as yf
from langchain.tools import tool
import functools

https://github.com/SALT-NLP/GenUI
https://www.youtube.com/watch?v=Vd2WLQ8vqfU
https://github.com/danduh/generative-ui
https://hashbrown.dev/
https://crayonai.org/

"You are a QUIZ GAME SHOW HOST running a trivia challenge tailored to user interests.\nThe quiz continues until the player makes a mistake, with difficulty increasing as their streak grows.\nAt the start, ask the user to select their hobbies, passions, and expertise via a CheckboxGroup, and use these to generate all questions.\n\n<QUIZ_RULES>\n1. Unlimited questions, ends on the first wrong answer. Difficulty rises with streak length.\n2. Each question must have exactly 4 answer options.\n3. Correct answer → celebrate → show trivia/fact → continue.\n4. Wrong answer → reveal correct answer + explanation → show final streak score + summary → display Restart Game button.\n5. No question repetition, even across restarts.\n6. Lifelines: 50-50 (reduce to 2 options), Skip Question, Hint (brief contextual clue).\n7. Trivia or facts must follow each answer, regardless of correctness.\n</QUIZ_RULES>\n\n<UI_RULES>\n1. Questions: bold text in primary font color. Supporting text in secondary font color.\n2. Options: always Buttons (never ListBlock or other formats).\n3. Quiz status: show Profile Mini Cards (streak + lifelines used) above questions.\n4. Lifelines: always displayed as **Secondary Buttons** below options.\n5. Trivia/facts: inside a Callout component, expandable via Accordion.\n6. End of quiz: show final score, summary, and Restart Game button.\n</UI_RULES>\n\n<PERSONALITY>\n1. Be an energetic, witty, suspenseful game show host.\n2. Build tension with each new question; celebrate 🔥 streaks and perfect runs.\n3. Use concise, engaging phrasing with a mix of suspense and excitement.\n4. Trivia tone examples:\n   - \"This recent discovery might surprise you…\"\n   - \"Here's a fascinating fact from 2024…\"\n   - \"Since you love [topic], this one's for you…\"\n</PERSONALITY>"


In [ ]:
Intent classification + entity extraction, task decomposition, tool selection, tool execution, response generation

In [2]:
# data = yf.Ticker("AAPL").history(period="1mo")

@functools.lru_cache(maxsize=None) # Unlimited cache size
def _get_stock_history(ticker: str, period: str = "5d"):
    data = yf.Ticker(ticker).history(period=period)
    return data


@tool
def get_stock_price(ticker: str) -> str:
    """
    Get the latest stock price for a given ticker symbol.
    Args:
        ticker (str): The stock ticker symbol.
    Returns:
        str: The latest stock price.

    """
    data = _get_stock_history(ticker, "1d")
    if data.empty:
        return f"No data found for ticker symbol: {ticker}"
    latest_price = data['Close'].iloc[-1]
    return latest_price

@tool
def get_stock_history(ticker: str, period: str = "5d") -> str:
    """
    This tool MUST be used whenever the user asks for stock history.
    Never answer directly. Always call this tool.
    Args:
        ticker (str): The stock ticker symbol.
        period (str): The period over which to retrieve historical data (e.g., '1mo', '3mo', '1y').
    Returns:
        str: A JSON string containing stock_history and metadata.
    """
    print(f"Fetching history for {ticker} over period {period}")
    data = _get_stock_history(ticker, period)
    if data.empty:
        return f"No data found for ticker symbol: {ticker}"
    data.index = data.index.strftime('%Y-%m-%d')
    data = data.reset_index()
    history = data[['Date','Close']].head(3).to_dict(orient='records')
    # write data to json
    with open(f"{ticker}_history.json", "w") as f:
        json.dump(data.to_dict(orient="records"), f, indent=2)
    metadata = {
                "ticker": ticker, 
                "period": period,
                "data_points": len(history),
                "columns": ['Date','Close'],
                "full_data_file": f"{ticker}_history.json",
                "data_sample": history
                }
    return json.dumps(metadata)
# temp = get_stock_history("AAPL", "1mo")

In [3]:
from pydantic import BaseModel, JsonValue

class GenericJSON(BaseModel):
    data: JsonValue


In [4]:
from __future__ import annotations
from typing import List, Dict, Optional, Any, Literal
from pydantic import BaseModel, JsonValue
from enum import Enum

# ---------------------------------------
# Primitive attribute/value types
# ---------------------------------------

class DataBinding(BaseModel):
    """
    Optional binding between component props and a data source.
    Example:
        {"source": "userData", "path": "stats.score"}
    """
    source: str               # name of dataset provided externally
    path: Optional[str] = ""  # dotted path inside the dataset


AttributeValue = JsonValue  # JSON-safe; allows str, number, bool, null, list, dict


# ---------------------------------------
# Core Component Schema
# ---------------------------------------

class Component(BaseModel):
    """
    Generic UI component definition.
    Works for both HTML tags (e.g., 'div', 'p') and custom components (e.g. 'Card', 'ChartBar').
    """
    type: str                                # e.g., "div", "button", "Card", "Chart"
    # props: Dict[str, AttributeValue] = {}    # HTML attributes or component props
    # style: Dict[str, AttributeValue] = {}    # Inline CSS-like style dictionary
    # events: Dict[str, str] = {}              # e.g. {"onClick": "handleClick"}
    # bind: Optional[Dict[str, DataBinding]] = None  # e.g. {"value": DataBinding(...)}
    children: Optional[List["Component"]] = None   # nested components


# ---------------------------------------
# Top-Level SpecSheet Schema
# ---------------------------------------

class SpecSheet(BaseModel):
    """
    Specification for HTML or Web Component rendering.
    """
    root: Component
    # metadata: Dict[str, Any] = {}            # Optional: author, timestamps, source LLM, etc.
    # datasets: Dict[str, JsonValue] = {}      # Optional: dynamic data made available to components

class Tags(Enum):
    DIV = "div"
    P = "p"
    SPAN = "span"
    BUTTON = "button"
    INPUT = "input"
    FORM = "form"
    TABLE = "table"
    TR = "tr"
    TD = "td"
    UL = "ul"
    LI = "li"
    H1 = "h1"
    H2 = "h2"
    H3 = "h3"
    H4 = "h4"
    H5 = "h5"
    H6 = "h6"
    IMG = "img"
    A = "a"

class HTMLSchema(BaseModel):
    """
    Specification for rendering standard HTML.
    """
    tag: Tags
    content: str                         # e.g., "div", "p", "span"

In [33]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)


tools = [get_stock_price, get_stock_history]

structured_llm = llm.with_structured_output(GenericJSON)

llm_with_tools = llm.bind_tools(tools)

# ui_llm = llm.with_structured_output(SpecSheet)

In [31]:
from langchain_openai import ChatOpenAI

ui_generation_llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='gpt-4.1',
    temperature=0
)

In [34]:
llm

ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x352e8cc20>, default_metadata=(), model_kwargs={})

In [26]:
ui_generation_llm = ui_generation_llm.with_structured_output(GenericJSON)

In [7]:
# query = "Give me step by step instructions to make a cup of tea."
# query = "compare nvidia gtx 1660 ti and nvidia rtx 3060 in terms of performance, price and power consumption"
# query = "names of 10 top movie directors along with their top movie from 2000 to 2005"
query = "Get me the stock price history for Apple Inc. (AAPL) for the past month."

response = llm_with_tools.invoke(f"{query}")

In [8]:
type(response.model_dump_json())

str

In [9]:
response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_stock_history', 'arguments': '{"period": "1mo", "ticker": "AAPL"}'}, '__gemini_function_call_thought_signatures__': {'b9f0f270-1993-4787-8082-14aaaefadc88': 'CrQCAXLI2nxAyfoD8HxfeAlryluiC74QlLzLrQ+HVts1MHIHPa02g/Y17ZLFFf5mgpfVPnv5a2Tr7ZzdsH59U3SN0JV6BaSv1yn3fAI2ZFmW8x8os2YKtEszCc7nUdhogjEbGUvPog8DNXAOmbjAf7OWtcTKHO/tj/Xm+dbC8AmDT9U4Z5Z62YjdY0KAu8WuolGqP8dDvUlHZVx6HO7r1cW2xrvx5m4b0dXBm2rq5jsEbBpHXjXZRkfeaaEpVOxiyxd65nnaHsEUGJ7Fa/B8d9L1Bw+cX727t/UHFVFNOSpQ8mqn8obsQsK8x9YzXOkSHMWGH1eHQxSQO0xrQ6iQR1y5xKjNS/j6ErzI/k6tzeemJYpWhAYlZyd+mDv5Y9EfsZpXoo4JIwOc8GY7isA/qDQEL6tZqC4='}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--071767f6-5dd6-4cf2-9945-b3e884ba90ac-0', tool_calls=[{'name': 'get_stock_history', 'args': {'period': '1mo', 'ticker': 'AAPL'}, 'id': 'b9f0f

In [10]:
if "tool_calls" in response.model_dump():
    print("Tool calls made:")
    for call in response.model_dump()["tool_calls"]:
        print(call)

Tool calls made:
{'name': 'get_stock_history', 'args': {'period': '1mo', 'ticker': 'AAPL'}, 'id': 'b9f0f270-1993-4787-8082-14aaaefadc88', 'type': 'tool_call'}


In [11]:
if response.tool_calls:
    for tool_call in response.tool_calls:
        name = tool_call["name"]
        args = tool_call["args"]

        if name == "get_stock_history":
            # StructuredTool → call using .invoke()
            result = get_stock_history.invoke(args)
            print("Tool result:", result)

Fetching history for AAPL over period 1mo
Tool result: {"ticker": "AAPL", "period": "1mo", "data_points": 3, "columns": ["Date", "Close"], "full_data_file": "AAPL_history.json", "data_sample": [{"Date": "2025-11-10", "Close": 269.42999267578125}, {"Date": "2025-11-11", "Close": 275.25}, {"Date": "2025-11-12", "Close": 273.4700012207031}]}


In [12]:
json.dumps(result)

'"{\\"ticker\\": \\"AAPL\\", \\"period\\": \\"1mo\\", \\"data_points\\": 3, \\"columns\\": [\\"Date\\", \\"Close\\"], \\"full_data_file\\": \\"AAPL_history.json\\", \\"data_sample\\": [{\\"Date\\": \\"2025-11-10\\", \\"Close\\": 269.42999267578125}, {\\"Date\\": \\"2025-11-11\\", \\"Close\\": 275.25}, {\\"Date\\": \\"2025-11-12\\", \\"Close\\": 273.4700012207031}]}"'

In [13]:
query = f"""
You are a UI Component Planner. You are responsible for generating SpecSheets based on provided data object in JSON format.
Return only the SpecSheet JSON without any additional explanations or comments. 
Make sure to return the json with 'root' as the top-level key.

You can use use custom components like 'Card', 'LineChart', 'BarChart', etc., or standard HTML tags like 'div', 'p', 'table', etc. depending 
on the data and metadata provided. Strictly use data sample only to decide the structure of the components and not to fill in all the data.

Following is the data object you need to visualize:
{json.dumps(result)}

Example SpecSheet Output:
{{
  "root": {{
    "type": "div",
    "children": [
      {{
        "type": "h1",
        "children": [{{ "type": "text", "value": "Title" }}]
      }},
      {{
        "type": "p",
        "children": [{{ "type": "text", "value": "This is a paragraph." }}]
      }},
      {{
        "type": "LineChart",
        "props": {{
          "id": "line_chart_1",
          "data_source": "some_data_source as a file or url", 
          "title": "Sample Line Chart",
          }}

      }}
      {{
      "table": {{
        "type": "table",
        "children": [
          {{
            "type": "tr",
            "children": [
              {{ "type": "th", "children": [{{ "type": "text", "value": "Column 1" }}] }},
              {{ "type": "th", "children": [{{ "type": "text", "value": "Column 2" }}] }}
            ]
          }},
          {{
            "type": "tr",
            "children": [
              {{ "type": "td", "children": [{{ "type": "text", "value": "Data 1" }}] }},
              {{ "type": "td", "children": [{{ "type": "text", "value": "Data 2" }}] }}
            ]
          }}
        ]
      }}
    ]
  }}
}}

"""
print(query)


You are a UI Component Planner. You are responsible for generating SpecSheets based on provided data object in JSON format.
Return only the SpecSheet JSON without any additional explanations or comments. 
Make sure to return the json with 'root' as the top-level key.

You can use use custom components like 'Card', 'LineChart', 'BarChart', etc., or standard HTML tags like 'div', 'p', 'table', etc. depending 
on the data and metadata provided. Strictly use data sample only to decide the structure of the components and not to fill in all the data.

Following is the data object you need to visualize:
"{\"ticker\": \"AAPL\", \"period\": \"1mo\", \"data_points\": 3, \"columns\": [\"Date\", \"Close\"], \"full_data_file\": \"AAPL_history.json\", \"data_sample\": [{\"Date\": \"2025-11-10\", \"Close\": 269.42999267578125}, {\"Date\": \"2025-11-11\", \"Close\": 275.25}, {\"Date\": \"2025-11-12\", \"Close\": 273.4700012207031}]}"

Example SpecSheet Output:
{
  "root": {
    "type": "div",
    "ch

In [14]:
query = f"""
You are a UI Component Planner. Your task is to generate a single coherent SpecSheet JSON structure that visualizes the provided data object.

### OUTPUT FORMAT RULES
- Return **only** the SpecSheet JSON.
- The top-level key **must** be `"root"`.
- Do **not** include explanations, comments, Markdown, or prose.

### COMPONENT SELECTION RULES
- Choose **the minimal set of components** needed to meaningfully visualize the dataset.
- If a chart (e.g., LineChart, BarChart, Card) fully represents the data pattern, **do NOT generate a table or redundant representations**.
- Only include additional components if they introduce **non-redundant information** (e.g., headers, summary cards).
- Use the provided `data_sample` **only to infer structure**, not to generate full tables or replicate all values.

### STRUCTURING RULES
- Use `Card`, `LineChart`, `BarChart`, or standard HTML tags (`div`, `p`, `table`, etc.) as appropriate.
- Base your component choice strictly on:
  - the schema (`columns`)
  - data type patterns
  - metadata such as `period`, `ticker`, `full_data_file`
- Never infer or fabricate new data.

### INPUT DATA
Here is the data object you must visualize:
{json.dumps(result)}

### Example SpecSheet Output:
{{
  "root": {{
    "type": "div",
    "children": [
      {{
        "type": "h1",
        "children": [{{ "type": "text", "value": "Title" }}]
      }},
      {{
        "type": "p",
        "children": [{{ "type": "text", "value": "This is a paragraph." }}]
      }},
      {{
        "type": "LineChart",
        "props": {{
          "id": "line_chart_1",
          "data_source": "some_data_source as a file or url", 
          "title": "Sample Line Chart",
          "x_axis": "Infer from metadata"
          "y_axis":"Infer from metadata
          }}

      }}
      {{
      "table": {{
        "type": "table",
        "children": [
          {{
            "type": "tr",
            "children": [
              {{ "type": "th", "children": [{{ "type": "text", "value": "Column 1" }}] }},
              {{ "type": "th", "children": [{{ "type": "text", "value": "Column 2" }}] }}
            ]
          }},
          {{
            "type": "tr",
            "children": [
              {{ "type": "td", "children": [{{ "type": "text", "value": "Data 1" }}] }},
              {{ "type": "td", "children": [{{ "type": "text", "value": "Data 2" }}] }}
            ]
          }}
        ]
      }}
    ]
  }}
}}
"""

In [35]:
html_response = llm.invoke("""
You are a UI Generation Agent that creates UI specifications based on user task and json data provided.
Generate a JSON-based UI specification that effectively represents the data and functionality required for each task.

following is the data:
[
  {
    "task_description": "Provide details of the top 5 products based on customer ratings",
    "tool_name": "get_top_n_selling_products",
    "args": {
      "n": 5
    },
    "result": "[{\"name\": \"Green\", \"price\": 72, \"description\": \"Key affect article away news paper family yourself pass game.\", \"image_url\": \"https://dummyimage.com/502x574\", \"units_sold\": 917}, {\"name\": \"Plant\", \"price\": 59, \"description\": \"Why information level think sister not model walk act to cover without suffer.\", \"image_url\": \"https://dummyimage.com/427x676\", \"units_sold\": 827}, {\"name\": \"Your\", \"price\": 29, \"description\": \"Since pay enjoy media collection director level put movement early somebody.\", \"image_url\": \"https://placekitten.com/916/102\", \"units_sold\": 771}, {\"name\": \"Quite\", \"price\": 72, \"description\": \"Little cup benefit newspaper point and.\", \"image_url\": \"https://placekitten.com/567/769\", \"units_sold\": 218}, {\"name\": \"Week\", \"price\": 75, \"description\": \"From check window tell audience question.\", \"image_url\": \"https://dummyimage.com/755x125\", \"units_sold\": 150}]"
  }
]

Use the following UI components in your specification:

{
    "type": "root",
    "usage": "Use 'root' as the top-level container for the entire UI.",
    "children": [ ...components ]
},
{
  "type": "container",
  "usage": "Use 'container' to group related components together.",
  "direction": "vertical" | "horizontal",
  "children": [ ...components ]
},
{
  "type": "text",
  "variant": "header" | "subheader" | "paragraph",
  "usage": "Use 'text' to display static text content.",
  "value": "string"
},
{
  "type": "chart",
  "usage": "Use 'chart' to visualize data in different formats.",
  "variant": "line" | "bar" | "pie",
  "title": "string",
  "dataKey": "chart_data_key"
},
{
  "type": "table",
  "usage": "Use 'table' to display tabular data. Depending on the data, you can have different columns and rows.",
  "title": "string",
  "columns": ["col1", "col2"],
  "rows": [...],
  "dataKey": "table_stock_data"
},
{
  "type": "card",
  "usage": "Use 'card' to present information in a concise format.",
  "title": "string",
  "content": [ ...components ]
},
{
  "type": "button",
  "usage": "Use 'button' to trigger actions or events.",
  "label": "string",
  "action": "string"
},
{
  "type": "image",
  "usage": "Use 'image' to display visual content.",
  "src": "image_url",
  "alt": "string"
},
{
  "type": "list",
  "usage": "Use 'list' to display a collection of items.",
  "items": [ ...components ],
}

Respond with a JSON structure that outlines the UI layout, components, and their configurations.
Make sure to organize the UI components in a logical manner that enhances user experience and accessibility.
""")

In [36]:
html_response

AIMessage(content='```json\n{\n  "type": "root",\n  "children": [\n    {\n      "type": "container",\n      "direction": "vertical",\n      "children": [\n        {\n          "type": "text",\n          "variant": "header",\n          "value": "Top 5 Products by Units Sold"\n        },\n        {\n          "type": "list",\n          "items": [\n            {\n              "type": "card",\n              "title": "Green",\n              "content": [\n                {\n                  "type": "container",\n                  "direction": "vertical",\n                  "children": [\n                    {\n                      "type": "image",\n                      "src": "https://dummyimage.com/502x574",\n                      "alt": "Green product image"\n                    },\n                    {\n                      "type": "text",\n                      "variant": "subheader",\n                      "value": "Price: $72"\n                    },\n                    {\n     

In [37]:
print(html_response.content)

```json
{
  "type": "root",
  "children": [
    {
      "type": "container",
      "direction": "vertical",
      "children": [
        {
          "type": "text",
          "variant": "header",
          "value": "Top 5 Products by Units Sold"
        },
        {
          "type": "list",
          "items": [
            {
              "type": "card",
              "title": "Green",
              "content": [
                {
                  "type": "container",
                  "direction": "vertical",
                  "children": [
                    {
                      "type": "image",
                      "src": "https://dummyimage.com/502x574",
                      "alt": "Green product image"
                    },
                    {
                      "type": "text",
                      "variant": "subheader",
                      "value": "Price: $72"
                    },
                    {
                      "type": "text",
                     

In [23]:
json_obj = json.loads(html_response.data)
json_obj

TypeError: the JSON object must be str, bytes or bytearray, not int

In [19]:
def json_to_html(node):
    if node["type"] == "text":
        return node.get("value", "")
    tag = node["type"]
    children = node.get("children", [])
    inner_html = "".join(json_to_html(child) for child in children)
    return f"<{tag}>{inner_html}</{tag}>"

import json
# data = json.loads(json_obj)
html = json_to_html(json_obj["root"])
# print(html)

In [20]:
from IPython.display import display, HTML, JSON
display(HTML(html))

In [ ]:

spec = {
  "type": "root",
  "children": [
    {
      "type": "container",
      "direction": "vertical",
      "children": [
        {
          "type": "text",
          "variant": "header",
          "value": "Top 5 Products by Units Sold"
        },
        {
          "type": "list",
          "items": [
            {
              "type": "card",
              "title": "Green",
              "content": [
                {
                  "type": "container",
                  "direction": "vertical",
                  "children": [
                    {
                      "type": "image",
                      "src": "https://dummyimage.com/502x574",
                      "alt": "Green product image"
                    },
                    {
                      "type": "text",
                      "variant": "subheader",
                      "value": "Price: $72"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Units Sold: 917"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Description: Key affect article away news paper family yourself pass game."
                    }
                  ]
                }
              ]
            },
            {
              "type": "card",
              "title": "Plant",
              "content": [
                {
                  "type": "container",
                  "direction": "vertical",
                  "children": [
                    {
                      "type": "image",
                      "src": "https://dummyimage.com/427x676",
                      "alt": "Plant product image"
                    },
                    {
                      "type": "text",
                      "variant": "subheader",
                      "value": "Price: $59"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Units Sold: 827"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Description: Why information level think sister not model walk act to cover without suffer."
                    }
                  ]
                }
              ]
            },
            {
              "type": "card",
              "title": "Your",
              "content": [
                {
                  "type": "container",
                  "direction": "vertical",
                  "children": [
                    {
                      "type": "image",
                      "src": "https://placekitten.com/916/102",
                      "alt": "Your product image"
                    },
                    {
                      "type": "text",
                      "variant": "subheader",
                      "value": "Price: $29"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Units Sold: 771"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Description: Since pay enjoy media collection director level put movement early somebody."
                    }
                  ]
                }
              ]
            },
            {
              "type": "card",
              "title": "Quite",
              "content": [
                {
                  "type": "container",
                  "direction": "vertical",
                  "children": [
                    {
                      "type": "image",
                      "src": "https://placekitten.com/567/769",
                      "alt": "Quite product image"
                    },
                    {
                      "type": "text",
                      "variant": "subheader",
                      "value": "Price: $72"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Units Sold: 218"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Description: Little cup benefit newspaper point and."
                    }
                  ]
                }
              ]
            },
            {
              "type": "card",
              "title": "Week",
              "content": [
                {
                  "type": "container",
                  "direction": "vertical",
                  "children": [
                    {
                      "type": "image",
                      "src": "https://dummyimage.com/755x125",
                      "alt": "Week product image"
                    },
                    {
                      "type": "text",
                      "variant": "subheader",
                      "value": "Price: $75"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Units Sold: 150"
                    },
                    {
                      "type": "text",
                      "variant": "paragraph",
                      "value": "Description: From check window tell audience question."
                    }
                  ]
                }
              ]
            }
          ]
        }
      ]
    }
  ]
}